In [1]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt
from dataset import PINNDataset

if torch.cuda.is_available():
    device = "cuda"

%matplotlib inline

In [2]:
img_dir = os.path.join("data", "images")
dataset = PINNDataset(img_dir=img_dir)

dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

# Display image and label.
train_features, train_labels = next(iter(dataloader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]



Feature batch shape: torch.Size([32, 290, 2])
Labels batch shape: torch.Size([32])
